In [ ]:
import json,csv,time
import pandas as pd
from urllib.request import urlopen
from urllib import error
from datetime import date, datetime, timedelta

In [ ]:
# API of GDAX order book, see https://docs.gdax.com/#get-product-order-book
api_url = "https://api.gdax.com/products/BTC-USD/book?level=3"

# Get data every 1 mins(60s). 
timeout = 60

In [ ]:
# This function runs ~8s in my computer 
def get_data():
    check = True
    while check:
        try:
            network = json.loads(urlopen(api_url).read())
            flag = True
            check = False
            #print ('Network is OK.')
            pass
        except:
            print ("Network or server issue. wait 5 min to restart" )
            time.sleep(300)
            print ("Restart...")
            
    while flag:
        timestamp = int(time.time())
        if timestamp%60 == 0:
            flag = False
        else:
            timestamp = int(time.time())
            
    bookdata = json.loads(urlopen(api_url).read())
    timestr = datetime.fromtimestamp(timestamp).strftime("%Y%m%d_%H%M%S")

    bids = bookdata["bids"]
    asks = bookdata["asks"]
    all_data = list(map(list, zip(*(bids + asks))))

    df= pd.DataFrame(columns=["buy_indicator", "market_size", "price"])
    df["price"] = all_data[0]
    df["market_size"] = all_data[1]

    df.loc[0:len(bids),("buy_indicator")] = 1
    df.loc[len(bids):, ("buy_indicator")] = 0
    #print (len(bids),len(asks))

    df.to_csv('bitcoin_'+timestr+'.txt', index=False, sep=' ', 
                header=["buy_indicator", "market_size", "price"])

In [ ]:
# Execution
count = 0
while True :
    count += 1
    start_time = time.time()
    get_data()
    print (str(count) + ' Data File has been downloaded!')
    sleept = timeout-(time.time() - start_time)
    if sleept > 1:
        time.sleep(sleept)
        timestamp += timeout
    